# **Forecasting the Impact of Climate Change on Great Britain’s Offshore Wind Generation using Deep Learning**

### Group 3:
**Candidate Codes:** (KXBS7, PNXF3, NQMJ9, NCJB4, MMVQ8, KVQS4)

## **Table of Content**

1. Introduction
2. Data Sources
3. Data Extraction
4. Data Preprocessing
5. Baseline Modelling
6. Model Development and Evaluation
7. Hyperparameter Tuning
8. Model application

## **Introduction**
Climate change is exerting a significant influence on global weather patterns, as evidenced by the increasing frequency of heatwaves, wildfires, and other extreme weather events. As renewable energy has become central to decarbonization strategies within the power sector, it is critical to understand the long-term impacts of climate change on the performance of renewable energy assets, particularly given that changes in climate patterns can directly affect their energy output.

The trajectory of future climate conditions depends largely on present-day energy policies and the rate of technological advancement. Several climate models have been developed to project future scenarios based on varying assumptions. Among these, the Representative Concentration Pathways (RCPs), developed by the climate science community, provide standardized scenarios that relate different policy choices to levels of radiative forcing.

The aim of this study is to evaluate the impacts of climate change on offshore wind generation in Great Britain under different RCP scenarios. This study focuses on offshore wind in particular as it constitutes a major component of the renewable energy mix in Great Britain. Offshore wind farms have experienced rapid development due to the region’s favourable wind conditions and the advantages offshore sites offer in terms of consistent wind speeds.

A deep learning approach is applied in conducting this study under two specific scenarios:

1. Representative Concentration Pathway (RCP) 8.5 — a high-emission, business-as-usual scenario; and

2. Representative Concentration Pathway (RCP) 2.6 — a low-emission, best-case scenario.

The findings will enable policymakers and stakeholders to better evaluate the resilience of the United Kingdom’s offshore wind strategy and to identify necessary contingencies to maintain long-term energy security.

___


### **Project Aim**
To investigate the impact of climate change on Great Britain's offshore wind generation by developing a deep learning-based forecasting model under the RCP 2.6 and 8.5 scenarios. This analysis is conducted at the wind farm level to extract granular and actionable insights.

### **Objectives**
1. **Data Collection and Preprocessing:**  
Gather and preprocess granular wind farm-level data on offshore wind generation, historical meteorological variables, and future climate projections.

2. **Exploratory Data Analysis:**  
Analyze wind farm-specific data to identify key factors (local wind speeds, seasonal trends, and climate variations) influencing offshore wind generation.

3. **Model Development:**  
Develop a deep learning model capable of forecasting wind farm-level generation based on historical wind generation and climate data.

4. **Scenario-Based Forecasting:**  
Forecast future wind generation at individual wind farm sites under RCP 8.5 and RCP 2.6 climate scenarios using climate data projections.

5. **Impact Analysis:**  
Analyze the impact of climate change on individual wind farms, examining how different climate scenarios affect regional performance and the overall capacity of the offshore wind sector in Great Britain.

## **Methodology**
The approach this study takes is to train a deep learning model to predict wind generation based on historical wind generation and historical climate data. Then, the model is used to predict future wind generation under the RCP 2.6 and 8.5 scenarios using projected climate data for the respective scenarios.

The image below shows the detailed methodology.

![Methodology](Methodology.png)

## **Data Sources**
1. **Elexon** - Wind generation data per wind farm, from 2019 - 2024
2. **Climate Data Store, ERA5** - Historical climate data, 2019 - 2024
3. **Climate Data Store, CMIP5, CMIP6** - Project climate data under RCP 2.6 and RCP 8.5 scenarios, 2025 - 2045

## **Data Extraction**


### **Historical Climate Data Extraction**

The historical climate data is extracted from the "Climate Data Store" under the ERA5 model.

Data is extracted for 2019-2024 as that is the period for which historical wind generation data is extracted.

Note that certain wind farms act as multiple Balancing Mechanism Units (BMUs).

In [ ]:
# Import the required packages 
# If needed, first install the packages using "!pip install package_name" (replace package_name with the name of the package)
import cdsapi
import numpy as np
import pandas as pd
import requests
import csv
from io import StringIO
from datetime import datetime, timedelta
import os

In [ ]:
# Import the csv file containing all wind farm BMUs & locations
bmu_data_analysed = pd.read_csv("https://raw.githubusercontent.com/OkeMoyo/BENV0148_Group_3/main/Analysis%20of%20BMU%20not%20collecting.csv",
encoding="ISO-8859-1")

In [23]:
# Check the number of wind farms in the data
print("There are ", len(bmu_data_analysed), "total BMUs in the dataset.")

# Inpsect the data
bmu_data_analysed.head()

There are  66 total BMUs in the dataset.


,Unnamed: 0.1,Common Name,Settlement BMU ID,Data Retrieved?,REPD ID (New),Unnamed: 0,Ref ID,Operator (or Applicant),Site Name,Technology Type,...,Development Status (short),County,Region,Country,X-coordinate,Y-coordinate,Planning Permission Granted,Under Construction,Operational,Data Retrieved?.1
0,0,Burbo Bank Offshore Wind Farm,E_BURBO,YES,2539,2061,2539,Orsted (formerly Dong Energy),Burbo Bank Extension (Burbo Bank 2),Wind Offshore,...,Operational,Offshore,Offshore,England,315815,398892,10/12/2014,10/06/2016,27/04/2017,NaN
1,0,Burbo Bank Offshore Wind Farm,T_BRBEO-1,YES,2539,2061,2539,Orsted (formerly Dong Energy),Burbo Bank Extension (Burbo Bank 2),Wind Offshore,...,Operational,Offshore,Offshore,England,315815,398892,10/12/2014,10/06/2016,27/04/2017,NaN
2,2,Dudgeon Offshore Wind Farm,T_DDGNO-1,YES,2538,2060,2538,Statoil / Statkraft,Dudgeon East,Wind Offshore,...,Operational,Offshore,Offshore,England,575000,361000,06/07/2012,17/03/2016,15/10/2017,NaN
3,2,Dudgeon Offshore Wind Farm,T_DDGNO-2,YES,2538,2060,2538,Statoil / Statkraft,Dudgeon East,Wind Offshore,...,Operational,Offshore,Offshore,England,575000,361000,06/07/2012,17/03/2016,15/10/2017,NaN
4,2,Dudgeon Offshore Wind Farm,T_DDGNO-3,YES,2538,2060,2538,Statoil / Statkraft,Dudgeon East,Wind Offshore,...,Operational,Offshore,Offshore,England,575000,361000,06/07/2012,17/03/2016,15/10/2017,NaN


In [22]:
# Filter out the wind farms whose data were successfully extracted
bmu_data_filtered = bmu_data_analysed[bmu_data_analysed["Data Retrieved?"] == "YES"]
print(len(bmu_data_filtered), "of", len(bmu_data_analysed),"BMUs' data were successfully extracted from Elexon.")
bmu_data_filtered.head()

53 of 66 BMUs' data were successfully extracted from Elexon.


,Unnamed: 0.1,Common Name,Settlement BMU ID,Data Retrieved?,REPD ID (New),Unnamed: 0,Ref ID,Operator (or Applicant),Site Name,Technology Type,...,Development Status (short),County,Region,Country,X-coordinate,Y-coordinate,Planning Permission Granted,Under Construction,Operational,Data Retrieved?.1
0,0,Burbo Bank Offshore Wind Farm,E_BURBO,YES,2539,2061,2539,Orsted (formerly Dong Energy),Burbo Bank Extension (Burbo Bank 2),Wind Offshore,...,Operational,Offshore,Offshore,England,315815,398892,10/12/2014,10/06/2016,27/04/2017,NaN
1,0,Burbo Bank Offshore Wind Farm,T_BRBEO-1,YES,2539,2061,2539,Orsted (formerly Dong Energy),Burbo Bank Extension (Burbo Bank 2),Wind Offshore,...,Operational,Offshore,Offshore,England,315815,398892,10/12/2014,10/06/2016,27/04/2017,NaN
2,2,Dudgeon Offshore Wind Farm,T_DDGNO-1,YES,2538,2060,2538,Statoil / Statkraft,Dudgeon East,Wind Offshore,...,Operational,Offshore,Offshore,England,575000,361000,06/07/2012,17/03/2016,15/10/2017,NaN
3,2,Dudgeon Offshore Wind Farm,T_DDGNO-2,YES,2538,2060,2538,Statoil / Statkraft,Dudgeon East,Wind Offshore,...,Operational,Offshore,Offshore,England,575000,361000,06/07/2012,17/03/2016,15/10/2017,NaN
4,2,Dudgeon Offshore Wind Farm,T_DDGNO-3,YES,2538,2060,2538,Statoil / Statkraft,Dudgeon East,Wind Offshore,...,Operational,Offshore,Offshore,England,575000,361000,06/07/2012,17/03/2016,15/10/2017,NaN


In [24]:
# The datasets contain 53 BMUs, but how many of them are unique wind farms?
unique_names = bmu_data_filtered["Common Name"].unique()
print("There are", len(unique_names), "unique wind farms in the dataset")

There are 21 unique wind farms in the dataset


In [25]:
# Inspect the array of unique wind farms
unique_names

array(['Burbo Bank Offshore Wind Farm', 'Dudgeon Offshore Wind Farm',
       'Galloper Offshore Wind Farm',
       'Greater Gabbard Offshore Wind Farm',
       'Gwynt y Mor Offshore Wind Farm', 'Humber Offshore Wind Farm',
       'Lincs Offshore Wind Farm', 'London Array Wind Farm',
       'Ormonde Offshore Wind Farm', 'Race Bank Offshore Wind Farm',
       'Rampion Offshore Wind Farm', 'Sheringham Shoals Wind Farm',
       'Thanet Offshore Wind Farm', 'Walney Offshore Wind Farm',
       'Westermost Rough Wind Farm',
       'West of Duddon Sands Offshore Wind Farm',
       'Aberdeen Offshore Wind Farm', 'Beatrice Offshore Wind Farm',
       'East Anglia Offshore Wind Farm', 'Hornsea Offshore Wind Farm',
       'Triton Knoll Offshore Wind Farm'], dtype=object)

There are 21 unique offshore wind farms in the dataset for which data was able to be extracted from Elexon. This volume of data is acceptable for this project as not all wind farms participate in the UK electricity balancing market.

The x and y coordintaes of the wind farm locations in csv file are in the British National Grid (OSGB36) coordinate system, representing eastings (X) and northings (Y), respectively.

In [27]:
# Extract the x and y coordinates for each unique wind farm in the filtered dataset

for name in unique_names:
    # Get the first occurrence of this Common Name
    row = bmu_data_filtered[bmu_data_filtered["Common Name"] == name].iloc[0]
    
    # Extract coordinates
    x_coord = row['X-coordinate']
    y_coord = row['Y-coordinate']
    
    # Print the result
    print(f"{name} is located at: X = {x_coord} eastings, Y = {y_coord} northings")

Burbo Bank Offshore Wind Farm is located at: X = 315815 eastings, Y = 398892 northings
Dudgeon Offshore Wind Farm is located at: X = 575000 eastings, Y = 361000 northings
Galloper Offshore Wind Farm is located at: X = 678139 eastings, Y = 227875 northings
Greater Gabbard Offshore Wind Farm is located at: X = 670237 eastings, Y = 231640 northings
Gwynt y Mor Offshore Wind Farm is located at: X = 292082 eastings, Y = 396482 northings
Humber Offshore Wind Farm is located at: X = 566400 eastings, Y = 390760 northings
Lincs Offshore Wind Farm is located at: X = 567020 eastings, Y = 368144 northings
London Array Wind Farm is located at: X = 642078 eastings, Y = 197218 northings
Ormonde Offshore Wind Farm is located at: X = 306000 eastings, Y = 466842 northings
Race Bank Offshore Wind Farm is located at: X = 573300 eastings, Y = 363000 northings
Rampion Offshore Wind Farm is located at: X = 528790 eastings, Y = 83836 northings
Sheringham Shoals Wind Farm is located at: X = 610671 eastings, Y 

To extract the historical climate data for each unique wind farm, the coordinates are transformed from the BNG system to the WGS84 system for compatibility with the requirements of the climate data store. A function is created to generate bounding boxes specifying the location of the wind farms in the WGS84 coordinate system to enable data extraction from the climate data store.

In [28]:
# bounding box function that creates bounding box depending on windfarm location:
from pyproj import Transformer
# Create transformer: BNG (EPSG:27700) → WGS84 (EPSG:4326)
transformer = Transformer.from_crs("epsg:27700", "epsg:4326", always_xy=True)

def create_bounding_box(x_coord, y_coord):
   
    # Transform to (lon, lat)
    lon, lat = transformer.transform(x_coord, y_coord)
   
    # CMIP6 has a resolution of approximately 1.25 degrees
    era5_resolution = 0.28125
   
    # Create box centered on the point with size of one CMIP grid cell
    # Return box as [North, West, South, East]
    return [
        lat + (era5_resolution/2),  # North
        lon - (era5_resolution/2),  # West
        lat - (era5_resolution/2),  # South
        lon + (era5_resolution/2)   # East
    ]

References for spatial resolution used above:

https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation#heading-Spatialgrid

https://confluence.ecmwf.int/display/CKB/ERA5%3A+What+is+the+spatial+reference


In [ ]:
# Extract the historical climate data (2019-2024) for each wind farm using the climate data store api and the bounding box function
# Change all file paths below to your preferred directory
"""
client = cdsapi.Client()
from math import cos, radians

# Create a directory to store the data
if not os.path.exists('era5_data'):
    os.makedirs('era5_data')

# Create a directory to store the data
output_dir = "C:/Users/Oke/Documents/UCL MSc ESDA/ESDA_Term 2/BENV0148 Advanced Machine Learning/BENV0148 Coursework/ERA5 Historical Data"

# Dataset name
dataset = "reanalysis-era5-single-levels"

# Base request template
base_request = {
    "product_type": ["reanalysis"],
    "variable": [
        "10m_u_component_of_wind",
        "10m_v_component_of_wind",
        "2m_dewpoint_temperature",
        "2m_temperature",
        "surface_pressure"
    ],
    "year": ["2019", "2020", "2021", "2022", "2023", "2024"],
    "month": [
        "01", "02", "03", "04", "05", "06",
        "07", "08", "09", "10", "11", "12"
    ],
    "day": [
        "01", "02", "03", "04", "05", "06",
        "07", "08", "09", "10", "11", "12",
        "13", "14", "15", "16", "17", "18",
        "19", "20", "21", "22", "23", "24",
        "25", "26", "27", "28", "29", "30", "31"
    ],
    "time": [
        "00:00", "01:00", "02:00", "03:00", "04:00", "05:00",
        "06:00", "07:00", "08:00", "09:00", "10:00", "11:00",
        "12:00", "13:00", "14:00", "15:00", "16:00", "17:00",
        "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"
    ],
    "format": "grib"  # Using NetCDF format as requested - can also use NetCDF ???
}

# Process each unique wind farm
for farm_name in unique_names:
    # Get the first row with this farm name
    row = bmu_data_filtered[bmu_data_filtered['Common Name'] == farm_name].iloc[0]
    
    # Extract coordinates
    x_coord = row['X-coordinate']
    y_coord = row['Y-coordinate']
    
    # Skip if coordinates are missing
    if pd.isna(x_coord) or pd.isna(y_coord):
        print(f"Skipping {farm_name} due to missing coordinates")
        continue
    
    print(f"Processing {farm_name} at coordinates ({x_coord}, {y_coord})")
    
    # Create a bounding box at ERA5 native resolution
    area_box = create_bounding_box(x_coord, y_coord)
    print(f"  ERA5 bounding box: {area_box}")
    
    # Create a custom request for this farm
    farm_request = base_request.copy()
    farm_request["area"] = area_box
    
    # Set output filename - clean up farm name for file safety
    safe_farm_name = ''.join(c if c.isalnum() else '_' for c in farm_name)
    target = os.path.join(output_dir, f"{safe_farm_name}.grib")
    
    try:
        print(f"Downloading ERA5 data for {farm_name}...")
        client.retrieve(dataset, farm_request).download(target)
        print(f"Successfully downloaded data to {target}")
    except Exception as e:
        print(f"Error downloading data for {farm_name}: {str(e)}")

print("Processing complete!")
"""